<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Conversion-of-Kurt-Aland's-Synopsis-data" data-toc-modified-id="Conversion-of-Kurt-Aland's-Synopsis-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Conversion of Kurt Aland's Synopsis data</a></span><ul class="toc-item"><li><span><a href="#First-splitting-the-NT-files-into-single-NT-book-files" data-toc-modified-id="First-splitting-the-NT-files-into-single-NT-book-files-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>First splitting the NT files into single NT book files</a></span></li></ul></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

# Conversion of Kurt Aland's Synopsis data

In [2]:
import pandas as pd
import re
import numpy as np

I am using this code with some additions:

```python
import pandas as pd

# Create a sample DataFrame with your data
data = {'Column1': ['1:2-7', '1:18-25', '2:1-12']}
df = pd.DataFrame(data)

# Function to expand the ranges
def expand_ranges(entry):
    parts = entry.split(':')
    prefix = parts[0]
    range_part = parts[1]
    start, end = map(int, range_part.split('-'))
    num_digits = len(range_part.split('-')[1])
    return [f"{prefix}:{str(i).zfill(num_digits)}" for i in range(start, end + 1)]

# Apply the function to the DataFrame
df['Expanded'] = df['Column1'].apply(expand_ranges)

# Explode the list of expanded values into separate rows
df = df.explode('Expanded').reset_index(drop=True)

# Rename the columns as needed
df = df.rename(columns={'Expanded': 'NewColumn'})

# Display the resulting DataFrame
print(df)
```

## First splitting the NT files into single NT book files

In [2]:
import pandas as pd
 
# read DataFrame
Synopsis = pd.read_excel("/Users/oliverglanz/Library/CloudStorage/OneDrive-AndrewsUniversity/1200_AUS-research/Fabric-TEXT/0_data_N1904/N1904_Aland-Synopsis_v1.0.xlsx", "Base")
#SynopsisMT = pd.read_csv("/Users/oliverglanz/Library/CloudStorage/OneDrive-AndrewsUniversity/1200_AUS-research/Fabric-TEXT/0_data_N1904/N1904_Aland-Synopsis.txt", sep='\t')
# NA1904 = pd.read_excel("/Users/glanz/Library/CloudStorage/OneDrive-AndrewsUniversity/1200_AUS-research/Fabric-TEXT/0_data_NA1904/Nestle1904_source_v1.5_20221124.xlsx")


In [45]:
pd.set_option('display.max_columns', 100)
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23
2,3,II,Birth and Childhood,2,The Annunciation,NaN,NaN,1:26-38,NaN,4.0,24
3,4,II,Birth and Childhood,2,Mary’s Visit to Elizabeth,NaN,NaN,1:39-56,NaN,5.0,24
4,5,II,Birth and Childhood,2,The Birth of John the Baptist,NaN,NaN,1:57-80,NaN,6.0,25


In [46]:
Synopsis.dtypes

orig order       int64
PART ROMAN      object
Unnamed: 2      object
part number      int64
Event           object
Matthew         object
Mark            object
Luke            object
John            object
Section        float64
Page            object
dtype: object

In [47]:
# Function to extract numeric part from a string
def extract_numeric(string):
    return int(re.search(r'\d+', string).group())

# Function to expand the ranges
def expand_ranges(entry):
    if pd.notna(entry):  # Check if the entry is not NaN
        parts = entry.split(':')
        prefix = parts[0]
        range_part = parts[1]

        start, end = None, None

        if '-' in range_part:
            start, end = map(extract_numeric, range_part.split('-'))
        else:
            start = end = extract_numeric(range_part)
        
        if start is not None and end is not None:
            num_digits = max(len(str(end)), 2)  # Ensure at least 2 digits
            return [f"{prefix}:{str(i).zfill(num_digits)}" for i in range(start, end + 1)]
        else:
            return [entry]  # Keep the original entry as-is for invalid ranges
    else:
        return [entry]  # Keep NaN values as-is

In [48]:
# Apply the function to the DataFrame
Synopsis['Matthew_expanded'] = Synopsis['Matthew'].apply(expand_ranges)
Synopsis.head()


,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,[1:01]
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,[nan]
2,3,II,Birth and Childhood,2,The Annunciation,NaN,NaN,1:26-38,NaN,4.0,24,[nan]
3,4,II,Birth and Childhood,2,Mary’s Visit to Elizabeth,NaN,NaN,1:39-56,NaN,5.0,24,[nan]
4,5,II,Birth and Childhood,2,The Birth of John the Baptist,NaN,NaN,1:57-80,NaN,6.0,25,[nan]


In [49]:
# Explode the list of expanded values into separate rows
Synopsis = Synopsis.explode('Matthew_expanded')
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,NaN
2,3,II,Birth and Childhood,2,The Annunciation,NaN,NaN,1:26-38,NaN,4.0,24,NaN
3,4,II,Birth and Childhood,2,Mary’s Visit to Elizabeth,NaN,NaN,1:39-56,NaN,5.0,24,NaN
4,5,II,Birth and Childhood,2,The Birth of John the Baptist,NaN,NaN,1:57-80,NaN,6.0,25,NaN


In [50]:
# Apply the function to the DataFrame
Synopsis['Mark_expanded'] = Synopsis['Mark'].apply(expand_ranges)
Synopsis.head()


,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded,Mark_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,[1:01]
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,NaN,[nan]
2,3,II,Birth and Childhood,2,The Annunciation,NaN,NaN,1:26-38,NaN,4.0,24,NaN,[nan]
3,4,II,Birth and Childhood,2,Mary’s Visit to Elizabeth,NaN,NaN,1:39-56,NaN,5.0,24,NaN,[nan]
4,5,II,Birth and Childhood,2,The Birth of John the Baptist,NaN,NaN,1:57-80,NaN,6.0,25,NaN,[nan]


In [51]:
# Explode the list of expanded values into separate rows
Synopsis = Synopsis.explode('Mark_expanded')
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded,Mark_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,NaN,NaN
2,3,II,Birth and Childhood,2,The Annunciation,NaN,NaN,1:26-38,NaN,4.0,24,NaN,NaN
3,4,II,Birth and Childhood,2,Mary’s Visit to Elizabeth,NaN,NaN,1:39-56,NaN,5.0,24,NaN,NaN
4,5,II,Birth and Childhood,2,The Birth of John the Baptist,NaN,NaN,1:57-80,NaN,6.0,25,NaN,NaN


In [52]:
# Apply the function to the DataFrame
Synopsis['Luke_expanded'] = Synopsis['Luke'].apply(expand_ranges)
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded,Mark_expanded,Luke_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,"[1:01, 1:02, 1:03, 1:04]"
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,NaN,NaN,"[1:05, 1:06, 1:07, 1:08, 1:09, 1:10, 1:11, 1:1..."
2,3,II,Birth and Childhood,2,The Annunciation,NaN,NaN,1:26-38,NaN,4.0,24,NaN,NaN,"[1:26, 1:27, 1:28, 1:29, 1:30, 1:31, 1:32, 1:3..."
3,4,II,Birth and Childhood,2,Mary’s Visit to Elizabeth,NaN,NaN,1:39-56,NaN,5.0,24,NaN,NaN,"[1:39, 1:40, 1:41, 1:42, 1:43, 1:44, 1:45, 1:4..."
4,5,II,Birth and Childhood,2,The Birth of John the Baptist,NaN,NaN,1:57-80,NaN,6.0,25,NaN,NaN,"[1:57, 1:58, 1:59, 1:60, 1:61, 1:62, 1:63, 1:6..."


In [53]:
# Explode the list of expanded values into separate rows
Synopsis = Synopsis.explode('Luke_expanded')
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded,Mark_expanded,Luke_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:02
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:03
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:04
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,NaN,NaN,1:05


In [54]:
# Apply the function to the DataFrame
Synopsis['John_expanded'] = Synopsis['John'].apply(expand_ranges)
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded,Mark_expanded,Luke_expanded,John_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01,"[1:01, 1:02, 1:03, 1:04, 1:05, 1:06, 1:07, 1:0..."
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:02,"[1:01, 1:02, 1:03, 1:04, 1:05, 1:06, 1:07, 1:0..."
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:03,"[1:01, 1:02, 1:03, 1:04, 1:05, 1:06, 1:07, 1:0..."
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:04,"[1:01, 1:02, 1:03, 1:04, 1:05, 1:06, 1:07, 1:0..."
1,2,II,Birth and Childhood,2,The Promise of the Birth of John the Baptist,NaN,NaN,1:5-25,NaN,3.0,23,NaN,NaN,1:05,[nan]


In [55]:
# Explode the list of expanded values into separate rows
Synopsis = Synopsis.explode('John_expanded')
Synopsis.head()

,orig order,PART ROMAN,Unnamed: 2,part number,Event,Matthew,Mark,Luke,John,Section,Page,Matthew_expanded,Mark_expanded,Luke_expanded,John_expanded
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01,1:01
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01,1:02
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01,1:03
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01,1:04
0,1,I,Preface,1,Prologue,1:1,1:1,1:1-4,1:1-18,NaN,21,1:01,1:01,1:01,1:05


In [56]:
Synopsis.to_excel("/Users/oliverglanz/Library/CloudStorage/OneDrive-AndrewsUniversity/1200_AUS-research/Fabric-TEXT/0_data_N1904/N1904_Aland-Synopsis_v1.1.xlsx")

# Testing

In [3]:
# First, I have to laod different modules that I use for analyzing the data and for plotting:
import sys, os, collections
import pandas as pd
import numpy as np
import re
import csv
import seaborn as sns
import matplotlib.pyplot as plt; plt.rcdefaults()
from matplotlib.pyplot import figure
from collections import Counter

# Second, I have to load the Text Fabric app
from tf.fabric import Fabric
from tf.app import use

In [7]:
#NA2 = use('NA1904/tf/1904_v3', hoist=globals())
#NA = use("CenterBLC/NA", version="1904")
NA = use("CenterBLC/NA:latest", version="1904")

**Locating corpus resources ...**

   |     0.69s T AlandSynopChapter    from ~/text-fabric-data/github/CenterBLC/NA/tf/1904
   |     0.70s T AlandSynopEvent      from ~/text-fabric-data/github/CenterBLC/NA/tf/1904
   |     0.65s T AlandSynopNo         from ~/text-fabric-data/github/CenterBLC/NA/tf/1904


Name,# of nodes,# slots/node,% coverage
book,27,5102.93,100
clause,27,5102.93,100
sentence,27,5102.93,100
chapter,260,529.92,100
verse,7943,17.35,100
word,137779,1.00,100


In [15]:
Search0 = '''
b1:book book=Matthew
 w1:word AlandSynopNo bol_tense* bol_lemma*
b2:book book=Mark
 w2:word AlandSynopNo bol_tense* bol_lemma*
 
w1 = w2

'''
Search0 = NA.search(Search0)
NA.show(Search0, start=1, end=3, condensed=True, colorMap={1:'pink'}, extraFeatures={})

  0.56s 0 results
